로지스틱 회귀가 선형 회귀와 다른 점은 학습을 통해 선형 함수의 회귀 최적선을 찾는 것이 아니라 <span style="color:red">시그모이드(Sigmoid) 함수 최적선</span>을 찾고 이 <span style="color:blue">시그모이드 함수의 반환 값을 확률로 간주해 확률에 따라 분류를 결정</span>한다는 것이다.

가볍고 빠르며, 이진 분류 예측 성능도 뛰어나 이진 분류의 기본 모델로 사용하는 경우가 많다. 희소한 데이터 세트 분류에서 뛰어난 성능을 보여서 텍스트 분류에서도 자주 사용된다.

시그모이드 함수 -> $y=\frac{1}{1+e^{-x}}$

<img src="https://media.vlpt.us/images/metterian/post/ee23f919-20f3-4acd-8110-b39a99df6096/image-20210413212222497.png">

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression

cancer = load_breast_cancer()

In [2]:
# 선형 회귀 계열의 로지스틱 회귀는 데이터의 정규 분포도에 따라 예측 성능 영향을 받을 수 있음.
# 따라서 데이터에 먼저 정규 분포 형태의 표준 스케일링을 적용

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# StandardScaler()로 평균이 0, 분산 1로 데이터 분포도 변환
scaler = StandardScaler()
data_scaled = scaler.fit_transform(cancer.data)

X_train, X_test, y_train, y_test = train_test_split(data_scaled, cancer.target, test_size=0.3, random_state=0)

In [3]:
from sklearn.metrics import accuracy_score, roc_auc_score

# 로지스틱 회귀를 이용해 학습 및 예측 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
lr_preds = lr_clf.predict(X_test)

# 정확도와 roc_auc 측정
print('accuracy: {:0.3f}'.format(accuracy_score(y_test, lr_preds)))
print('roc_auc: {:0.3f}'.format(roc_auc_score(y_test, lr_preds)))

accuracy: 0.977
roc_auc: 0.972


#### LogisticRegression 클래스의 주요 하이퍼 파라미터
- penalty: Regularization의 유형 설정, 'l2'와 'l1'이 있음. 기본은 'l2'
- C: 규제 강도를 조절하는 alpha 값의 역수. 즉, $C=\frac{1}{alpha}$

In [4]:
from sklearn.model_selection import GridSearchCV

params={'penalty':['l2', 'l1'],
       'C':[0.01, 0.1, 1, 5, 10]}

grid_clf = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=3)
grid_clf.fit(data_scaled, cancer.target)
print('최적 하이퍼 파라미터:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_clf.best_params_, grid_clf.best_score_))

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 449, in _check_solver
    % (solver, penalty)
ValueError: Solver

최적 하이퍼 파라미터:{'C': 1, 'penalty': 'l2'}, 최적 평균 정확도:0.975
